In [27]:
# TSP processing
# !pip install haversine
# !pip install protobuf==4.21
# !pip install tensorflow-metadata==1.14.0
# !pip install ortools



import pandas as pd
from haversine import haversine, Unit

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import time
import warnings
warnings.filterwarnings('ignore')


In [28]:
# need to create data model first
def create_data_model(df):
    """Stores the data for the problem."""
    data = {}
    depot_lat = 2.971718
    depot_lng = 101.608376

    _data = df[['lat', 'lng']]

    _data.loc[-1] = [depot_lat, depot_lng]  # adding a row
    _data.index = _data.index + 1           # shifting index
    _data.sort_index(inplace=True)

    results = []
    for i in range(len(_data)):
        lat = _data.iloc[i]['lat']
        lng = _data.iloc[i]['lng']
        from_node = (lat, lng)
        result = []
        for j in range(len(_data)):
            lat = _data.iloc[j]['lat']
            lng = _data.iloc[j]['lng']
            to_node = (lat, lng)
            if j == 0:
                distance = 0
            else:
                distance = round(haversine(from_node, to_node, unit=Unit.METERS))
            result.append(distance)
        results.append(result)

    data['distance_matrix'] = results
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


In [51]:
# running TSP process
def Running_TSP(_df, cluster_number):
    start_time = time.time()
    
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(_df)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        """Prints solution on console."""

        index = routing.Start(0)
        plan_output = 'Route for vehicle {}:\n'.format(cluster_number)
        route_distance = 0
        idx = 0
        _cluster = []
        _order = []
    
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            plan_output += ' {} ->'.format(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            
            _cluster.append(cluster_number)
            if node_index > 0:
                _order.append(int(idx))
            idx += 1
            
        plan_output += " {}\n".format(manager.IndexToNode(index))
        plan_output += "Route distance: {} km\n".format(route_distance/1000)
        plan_output += "Running_TSP --- {} seconds ---\n".format((time.time() - start_time))
        print(plan_output)
        
        _df['order'] = _order
        return route_distance

In [63]:
# -------------------------------------------------------
#
### This code to show the TSP from ORTools can be use ###
#
# -------------------------------------------------------

file_url = 'dataset/geolocation_result.csv'
_file = file_url.strip(".csv")
file_result = _file + '_tsp.csv'
data = pd.read_csv(file_url)

_job = data.groupby('cluster')['no'].count().sort_values(ascending=[False])
print(_job)

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

total_distance = 0
start_time_tsp = time.time()
for nc in range(len(_job)):
    _clusterid = _job.index[nc]
    
    _filter = (data['cluster'] == _clusterid)    
    _dataCluster = data.loc[_filter]
    
    
    route_distance = Running_TSP(_dataCluster, _clusterid)
    total_distance += route_distance / 1000
    
    for i in range(len(_dataCluster)):
        idx = _dataCluster.index[i]
        value = _dataCluster.iloc[i]
        data.loc[idx, 'order'] = int(value['order'])

print("All TSP--- %s seconds ---" % (time.time() - start_time_tsp))
print("Total Distance: %s km" % total_distance)

data['order'] = data['order'].astype(int)
data.to_csv(file_result)  


cluster
9     31
3     30
15    28
14    28
13    28
12    28
11    28
10    28
8     28
7     28
6     28
5     28
4     28
2     28
1     28
0     28
Name: no, dtype: int64
Route for vehicle 9:
 0 -> 22 -> 11 -> 31 -> 21 -> 9 -> 24 -> 19 -> 27 -> 17 -> 23 -> 1 -> 28 -> 12 -> 4 -> 10 -> 16 -> 14 -> 13 -> 2 -> 30 -> 25 -> 29 -> 26 -> 3 -> 6 -> 20 -> 18 -> 15 -> 8 -> 7 -> 5 -> 0
Route distance: 15.514 km
Running_TSP --- 0.39466404914855957 seconds ---

Route for vehicle 3:
 0 -> 24 -> 23 -> 29 -> 6 -> 12 -> 28 -> 25 -> 20 -> 11 -> 9 -> 1 -> 26 -> 4 -> 18 -> 17 -> 14 -> 7 -> 3 -> 16 -> 22 -> 19 -> 30 -> 21 -> 15 -> 5 -> 13 -> 8 -> 2 -> 27 -> 10 -> 0
Route distance: 17.476 km
Running_TSP --- 0.280623197555542 seconds ---

Route for vehicle 15:
 0 -> 14 -> 28 -> 11 -> 15 -> 7 -> 6 -> 18 -> 3 -> 16 -> 22 -> 17 -> 1 -> 2 -> 25 -> 12 -> 21 -> 20 -> 10 -> 13 -> 9 -> 5 -> 4 -> 24 -> 19 -> 27 -> 8 -> 23 -> 26 -> 0
Route distance: 12.573 km
Running_TSP --- 0.25290393829345703 seconds ---

Route f

In [67]:
import folium
from folium.features import DivIcon

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', \
     'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', \
     'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray', 'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', \
     'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', \
     'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray' ]

lat = data.iloc[10]['lat']
lng = data.iloc[10]['lng']
map = folium.Map(location=[lat, lng], zoom_start=12)

for _, row in data.iterrows():
    folium.CircleMarker(location=[row["lat"], row["lng"]],
                        radius=12, weight=2, fill=True, fill_color=colors[int(row["cluster"])], \
                        color=colors[int(row["cluster"])]).add_to(map)

    folium.Marker(location=[row["lat"], row["lng"]], icon=DivIcon(icon_size=(150,36), icon_anchor=(5,10),
        html='<div style="font-size: 10pt; color : {}">{}</div>' \
            .format(colors[int(row['cluster'])], int(row['order'])))).add_to(map)

map